In [ ]:
!pip install -q accelerate==0.33.0 bitsandbytes==0.43.3 transformers==4.43.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 48.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 117.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain import PromptTemplate
import traceback
from peft import PeftModel
from transformers import BitsAndBytesConfig
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from typing import List
from transformers import StoppingCriteria, StopStringCriteria,StoppingCriteriaList
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)
from langchain.memory import ConversationBufferWindowMemory
# from .history_chat import retrieve_history_chat
# from .vdb import find_similar_answer1
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


# Model/Tokenizer Loading
MODEL_NAME = "NousResearch/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto",
    quantization_config=bnb_config,
)

# Generation Configuration (adjust parameters as desired)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 528
generation_config.temperature = 0.01
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
generation_config.eos_token_id = tokenizer.eos_token_id
stop_strings = ["Patient", "AI Doctor"]  # Add the strings that should end the generation
stop_criteria = StopStringCriteria(tokenizer=tokenizer, stop_strings=stop_strings)
stopping_criteria = StoppingCriteriaList([stop_criteria])  # Add more criteria as needed
text_pipeline = pipeline(
    "text-generation",
    model=model,
    num_return_sequences=1,
    tokenizer=tokenizer,
    generation_config=generation_config,
    # eos_token_id=terminators,  # I already set the eos_token_id here, still no end for its self-coververstaion
    pad_token_id=tokenizer.eos_token_id,
    stopping_criteria=stopping_criteria,
    return_full_text=True,
    max_new_tokens=2048,
)
llm = HuggingFacePipeline(pipeline=text_pipeline)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

template = """The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
{history}
Patient: {input}
AI Doctor:"""
PROMPT = PromptTemplate(input_variables=["input", "history"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(human_prefix="Patient"),
)



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [ ]:
res = conversation.predict(input="I have a headache.")
print(res.split("AI Doctor")[1])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:

Patient: I have a headache.
AI Doctor:

> Finished chain.
: How long has this been going on?
Patient


In [ ]:
res = conversation.predict(input="for a week ")
print(res.split("AI Doctor")[2])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
Patient: I have a headache.
AI: The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms


In [ ]:
res = conversation.predict(input="6")
print(res.split("AI Doctor")[2])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
Patient: I have a headache.
AI: The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms


In [ ]:
print(res.split("AI Doctor")[4])

: Do you experience any other symptoms besides headaches?
Patient


In [ ]:
res = conversation.predict(input="untill nw no  ")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
Patient: I have a headache.
AI: The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms


In [ ]:
print(res.split("AI Doctor")[8])

: Have you had any recent illnesses or injuries that could be causing these symptoms?
Patient


In [ ]:
res = conversation.predict(input="untill nw no")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
Patient: I have a headache.
AI: The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms


In [ ]:
print(res.split("AI Doctor")[16])

: Are there any medications or supplements you're taking regularly?
Patient


In [ ]:
res = conversation.predict(input="no")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
Patient: I have a headache.
AI: The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms


In [ ]:
print(res.split("AI Doctor")[32])

: Is there anything else about yourself that might affect my ability to diagnose or treat you effectively?
Patient


In [ ]:
res = conversation.predict(input="i think no if u have anu questions u can ask me and i will answer ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms
- Medical history
- Lifestyle factors

You may also ask any other questions you think are necessary to provide a correct response to the patient.

Current conversation:
Patient: I have a headache.
AI: The following is a conversation between a patient and an AI doctor.

As the AI doctor, your goal is to help the patient with diagnosis or referral. You must ask all relevant questions before providing a diagnosis, treatment, or referral. These questions include:

- Duration of the disease
- Severity of symptoms on a scale of 1 to 10
- Pain level
- Other symptoms


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.81 GiB. GPU 